In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
num_classes = 5
img_rows, img_cols = 48, 48
batch_size = 32

In [4]:
train_data_dir = 'fer2013/train'
validation_data_dir = 'fer2013/validation'

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=30, 
    shear_range=0.3, 
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
train_data_dir, 
color_mode='grayscale',
target_size=(img_rows, img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
validation_data_dir, 
color_mode='grayscale',
target_size=(img_rows, img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True
)

Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.


In [14]:
model = tf.keras.applications.ResNet50(
    input_shape=(img_rows, img_cols, 1),
    weights=None,
    classes=num_classes)

In [15]:
print(model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 1)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 24, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [11]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [earlystop, checkpoint, reduce_lr]

In [17]:
model.compile(
loss='categorical_crossentropy',
optimizer=Adam(lr=0.001),
metrics=['accuracy'])
n_train_samples = 24176
n_val_samples = 3006
epochs = 25

In [18]:
history = model.fit(
train_generator,
steps_per_epoch=n_train_samples//batch_size,
epochs=epochs,
callbacks=callbacks,
validation_data=validation_generator,
validation_steps=n_val_samples//batch_size)

Epoch 1/25
754/755 [============================>.] - ETA: 0s - loss: 1.9459 - accuracy: 0.2696
Epoch 00001: val_loss improved from inf to 6.00605, saving model to EmotionDetectionModel.h5
755/755 [==============================] - 37s 50ms/step - loss: 1.9456 - accuracy: 0.2696 - val_loss: 6.0061 - val_accuracy: 0.2886
Epoch 2/25
755/755 [==============================] - ETA: 0s - loss: 1.6408 - accuracy: 0.2960
Epoch 00002: val_loss improved from 6.00605 to 2.07227, saving model to EmotionDetectionModel.h5
755/755 [==============================] - 37s 49ms/step - loss: 1.6408 - accuracy: 0.2960 - val_loss: 2.0723 - val_accuracy: 0.2332
Epoch 3/25
754/755 [============================>.] - ETA: 0s - loss: 1.6438 - accuracy: 0.2890
Epoch 00003: val_loss improved from 2.07227 to 1.62835, saving model to EmotionDetectionModel.h5
755/755 [==============================] - 37s 49ms/step - loss: 1.6441 - accuracy: 0.2891 - val_loss: 1.6284 - val_accuracy: 0.2940
Epoch 4/25
755/755 [======